In [80]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.metrics import cohen_kappa_score,make_scorer
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_log_error
#print('RMSLE:', sqrt(mean_squared_log_error(np.exp(y_cv), np.exp(y_pred2))))
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#X_train, X_test, y_train, y_test = train_test_split(X, label_encoded_y, test_size=test_size, random_state=seed)
#accuracy = accuracy_score(y_test, predictions)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
from sklearn.decomposition import PCA
from mlxtend.regressor import StackingCVRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from math import sqrt
from sklearn.svm import SVR


In [3]:
train=pd.read_excel('Data_Train.xlsx')
test=pd.read_excel('Data_Test.xlsx')
submission=pd.read_excel('Sample_submission.xlsx')

In [4]:
train.shape

(12690, 9)

In [5]:
length=len(train)
length

12690

In [6]:
test.shape

(4231, 8)

In [7]:
train_append=train.append(test)

In [8]:
train_append=train_append.reset_index(drop=True)

In [9]:
train_append.shape

(16921, 9)

In [10]:
train1=train_append.drop(['COST'],axis=1)

In [11]:
train1.head()

,CITY,CUISINES,LOCALITY,RATING,RESTAURANT_ID,TIME,TITLE,VOTES
0,Thane,"Malwani, Goan, North Indian",Dombivali East,3.6,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",CASUAL DINING,49 votes
1,Chennai,"Asian, Modern Indian, Japanese",Ramapuram,4.2,13198,6pm – 11pm (Mon-Sun),"CASUAL DINING,BAR",30 votes
2,Chennai,"North Indian, Chinese, Biryani, Hyderabadi",Saligramam,3.8,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",CASUAL DINING,221 votes
3,Mumbai,"Tibetan, Chinese",Bandra West,4.1,6346,11:30am – 1am (Mon-Sun),QUICK BITES,24 votes
4,Mumbai,Desserts,Lower Parel,3.8,15387,11am – 1am (Mon-Sun),DESSERT PARLOR,165 votes


In [12]:
train1.isnull().sum()

CITY              147
CUISINES            0
LOCALITY          128
RATING              4
RESTAURANT_ID       0
TIME                0
TITLE               0
VOTES            1606
dtype: int64

In [13]:
t=train1

In [14]:
t['c1'],t['c2'],t['c3'],t['c4'],t['c5'],t['c6'],t['c7'],t['c8']=t['CUISINES'].str.split(',',-1).str

In [15]:
t.head()

,CITY,CUISINES,LOCALITY,RATING,RESTAURANT_ID,TIME,TITLE,VOTES,c1,c2,c3,c4,c5,c6,c7,c8
0,Thane,"Malwani, Goan, North Indian",Dombivali East,3.6,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",CASUAL DINING,49 votes,Malwani,Goan,North Indian,NaN,NaN,NaN,NaN,NaN
1,Chennai,"Asian, Modern Indian, Japanese",Ramapuram,4.2,13198,6pm – 11pm (Mon-Sun),"CASUAL DINING,BAR",30 votes,Asian,Modern Indian,Japanese,NaN,NaN,NaN,NaN,NaN
2,Chennai,"North Indian, Chinese, Biryani, Hyderabadi",Saligramam,3.8,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",CASUAL DINING,221 votes,North Indian,Chinese,Biryani,Hyderabadi,NaN,NaN,NaN,NaN
3,Mumbai,"Tibetan, Chinese",Bandra West,4.1,6346,11:30am – 1am (Mon-Sun),QUICK BITES,24 votes,Tibetan,Chinese,NaN,NaN,NaN,NaN,NaN,NaN
4,Mumbai,Desserts,Lower Parel,3.8,15387,11am – 1am (Mon-Sun),DESSERT PARLOR,165 votes,Desserts,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
t['t1'],t['t2']=t['TITLE'].str.split(',',-1).str

In [17]:
t['votes'],t['v1']=t['VOTES'].str.split(' ',-1).str

In [18]:
t.isnull().sum()

CITY               147
CUISINES             0
LOCALITY           128
RATING               4
RESTAURANT_ID        0
TIME                 0
TITLE                0
VOTES             1606
c1                   0
c2                4047
c3                9330
c4               13309
c5               15550
c6               16370
c7               16671
c8               16813
t1                   0
t2               14977
votes             1606
v1                1606
dtype: int64

In [19]:
t['c2']=t['c2'].fillna('NA')
t['c3']=t['c3'].fillna('NA')
t['c4']=t['c4'].fillna('NA')
t['c5']=t['c5'].fillna('NA')
t['c6']=t['c6'].fillna('NA')
t['c7']=t['c7'].fillna('NA')
t['c8']=t['c8'].fillna('NA')
t['t2']=t['t2'].fillna('NA')


In [20]:
t1=t[['CITY','LOCALITY','RATING','RESTAURANT_ID','c1','c2','c3','c4','c5','c6','c7','c8','t1','t2','votes']]

In [21]:
t1['RATING']=t1['RATING'].replace('-',np.nan)
t1['RATING']=t1['RATING'].replace('NEW',np.nan)

In [22]:
t1.isnull().sum()

CITY              147
LOCALITY          128
RATING           1606
RESTAURANT_ID       0
c1                  0
c2                  0
c3                  0
c4                  0
c5                  0
c6                  0
c7                  0
c8                  0
t1                  0
t2                  0
votes            1606
dtype: int64

In [23]:
t1.shape

(16921, 15)

In [24]:
t11=t1[['RATING','votes']].astype(np.float64)
t12=t1[['LOCALITY','CITY']]

In [25]:
t12=pd.get_dummies(t12)

In [26]:
t12.shape

(16921, 2086)

In [27]:
from fancyimpute import IterativeImputer

Using TensorFlow backend.


In [28]:
mice=IterativeImputer( n_iter=10)
x1=mice.fit_transform(t11)
x2=mice.fit_transform(t12)

In [29]:
x1=pd.DataFrame(x1).round(1)
x2=pd.DataFrame(x2).round(1)

In [30]:
x1.head()

,0,1
0,3.6,49.0
1,4.2,30.0
2,3.8,221.0
3,4.1,24.0
4,3.8,165.0


In [31]:
x1.columns=[['RATING','votes']]

In [32]:
x2.head()

,0,1,2,3,4,5,6,7,8,9,...,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
s=list(t12.columns)



In [34]:
x2.columns=s

In [35]:
t1.isnull().sum()

CITY              147
LOCALITY          128
RATING           1606
RESTAURANT_ID       0
c1                  0
c2                  0
c3                  0
c4                  0
c5                  0
c6                  0
c7                  0
c8                  0
t1                  0
t2                  0
votes            1606
dtype: int64

In [36]:
t2=pd.get_dummies(t1[['c1','c2','c3','c4','c5','c6','c7','c8','t1','t2']])

In [37]:
t2.shape

(16921, 685)

In [38]:
t2.head()

,c1_Afghan,c1_African,c1_American,c1_Andhra,c1_Arabian,c1_Armenian,c1_Asian,c1_Assamese,c1_Awadhi,c1_BBQ,...,t2_IRANI CAFE,t2_KIOSK,t2_LOUNGE,t2_MEAT SHOP,t2_MICROBREWERY,t2_NA,t2_PAAN SHOP,t2_PUB,t2_QUICK BITES,t2_SWEET SHOP
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [39]:
t2.shape

(16921, 685)

In [40]:
t2.head()

,c1_Afghan,c1_African,c1_American,c1_Andhra,c1_Arabian,c1_Armenian,c1_Asian,c1_Assamese,c1_Awadhi,c1_BBQ,...,t2_IRANI CAFE,t2_KIOSK,t2_LOUNGE,t2_MEAT SHOP,t2_MICROBREWERY,t2_NA,t2_PAAN SHOP,t2_PUB,t2_QUICK BITES,t2_SWEET SHOP
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [41]:
x1.shape

(16921, 2)

In [42]:
x1.head()

,RATING,votes
0,3.6,49.0
1,4.2,30.0
2,3.8,221.0
3,4.1,24.0
4,3.8,165.0


In [43]:
x2.shape

(16921, 2086)

In [44]:
x2.head()

,LOCALITY_(Main Market) Mayur Vihar Phase 2,LOCALITY_(ground floor of Sunrise Chambers),LOCALITY_1 st Cross street,LOCALITY_1-7-172/1 Ecil Opposite,LOCALITY_100 Ft Road,LOCALITY_110009,LOCALITY_11th Cross Road,LOCALITY_14/90,LOCALITY_14th Main Rd HSR,LOCALITY_16,...,CITY_chullickal,CITY_hyderabad,CITY_khar west,CITY_kochi,CITY_navi mumbai,CITY_opp gurudwara Shakurpur,CITY_opp mukteshwar ashram powai,CITY_opposite to western side of ITPL Service gate,CITY_p.o Kochi,CITY_ward X11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
T=pd.concat([t2,x1,x2],axis=1)

In [46]:
T.shape

(16921, 2773)

In [47]:
final_train=T.iloc[:(length),:]

In [48]:
final_train.shape

(12690, 2773)

In [49]:
final_test=T.iloc[12690:,:]

In [50]:
final_test.shape

(4231, 2773)

In [51]:
y_train=train['COST']


In [52]:
y_train=np.asarray(y_train)

In [53]:
s=StandardScaler()
final_train1 = s.fit_transform(final_train)

final_test1 = s.transform(final_test)

Y_train = s.fit_transform(y_train.reshape(-1,1))



In [54]:
final_train1.shape

(12690, 2773)

In [55]:
final_test1.shape

(4231, 2773)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(final_train1, Y_train)

In [ ]:
from xgboost import XGBRegressor

In [495]:
xgb=XGBRegressor(learning_rate =0.01, 
      n_estimators=100, max_depth=12, min_child_weight=1,gamma=0, reg_alpha=2e-5,
      subsample=0.8,colsample_bytree=0.8,
      nthread=4,scale_pos_weight=1,seed=27,verbose=True,random_state=100)
xgb.fit(X_train,y_train) #Accuracy: 0.55

y =xgb.predict(X_test)


In [532]:
xgb=XGBRegressor()
xgb.fit(X_train,y_train) #Accuracy: 0.61

y =xgb.predict(X_test)


In [505]:
rf=RandomForestRegressor(n_estimators=1000, oob_score = False, n_jobs = -1,random_state =50,
                         max_features = "auto", min_samples_leaf = 2,warm_start=True,max_depth=50)
rf.fit(X_train,y_train) #Accuracy: 0.63

y =rf.predict(X_test)

In [543]:
rf=RandomForestRegressor()
rf.fit(X_train,y_train) #Accuracy:0.61

y =rf.predict(X_test)

In [502]:
gbr=GradientBoostingRegressor( loss = 'huber',learning_rate=0.001,n_estimators=350, max_depth=6
                              ,subsample=1,
                              verbose=False,random_state=126)   # Leaderboard SCORE :  0.8364249755816828 @ RS =126 ,n_estimators=350, max_depth=6

gbr.fit(X_train,y_train)

y =gbr.predict(X_test)


In [540]:
gbr=GradientBoostingRegressor()   

gbr.fit(X_train,y_train) #Accuracy: 0.61

y =gbr.predict(X_test)


In [528]:
y=pd.DataFrame(y)
print(y.shape)
y.head()

(4231, 1)


,0
0,991.132263
1,449.163300
2,621.795593
3,255.611130
4,297.436981


In [529]:
submission['COST']=y

In [530]:
submission['COST']=(submission['COST'].round(0))

In [531]:
submission.to_excel("d:/s/vs1.xlsx",index=False)